In [1]:
import os
import json
from tqdm import tqdm
from groq import Groq

In [2]:
client = Groq(
    api_key="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",
)

In [3]:
def get_response(json_content):
        chat_completion = client.chat.completions.create(
            model="mixtral-8x7b-32768",
            response_format = {"type": "json_object"},
            messages=[
                {
                    "role": "system",
                    "content": f"""
                    
        <s>[INST]
        Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  

        ### Instruction: Label the cases with an importance score based on the following criteria

        1 = High importance: All judgments, decisions, and advisory opinions not included in the
        Case Reports that make a significant contribution to the development, clarification or
        modification of its case law, either generally or in relation to a particular State.
        
        2 = Medium importance: Other judgments, decisions and advisory opinions which, while
        not making a significant contribution to the case-law, nevertheless go beyond merely
        applying existing case-law.        
        
        4 = Low importance: Judgments, decisions and advisory opinions of little legal interest,
        namely judgments and decisions that simply apply existing case-law, friendly
        settlements and strike outs (unless raising a particular point of interest).


        * Think step by step before responding. 
        * Check if the case just applies existing case law (low importance)
        * Check if the case imposes new clarification of the law (high importance)
        * Start your response by printing the importance score first as the label.
        * respond as a valid JSON Document

        ### Input:  
        Case:
        {json_content}

        ### Response:
        importance score:
        reason:

        [/INST]       
                    """,
                }
            ]
        )
        return chat_completion

In [4]:
# Directory paths
input_dir = './echr-prompts/'
output_dir = './zero-results/'

# Ensure directory exists
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)


In [14]:
# Process each JSON file
for filename in tqdm(os.listdir(input_dir)):
    if filename.endswith('.json'):
        file_path = os.path.join(input_dir, filename)
        
        # Load JSON file
        with open(file_path, 'r') as file:
            json_data = json.load(file)

        label = json_data['importance']
        del json_data['importance']
        del json_data['procedure']
        del json_data['law']
        
        # Convert JSON data to string
        json_string = json.dumps(json_data)
        
        # Get response
        response_content = get_response(json_string)
        response_dict = json.loads(response_content.choices[0].message.content)
        
        # Save response to output directory as a .json file
        data = {
            'label': label,
            'total tokens': response_content.usage.total_tokens,
            'completion_tokens': response_content.usage.completion_tokens,
            'prompt_tokens': response_content.usage.prompt_tokens,
            
        }
        data.update(response_dict)
        output_file_path = os.path.join(output_dir, f'response_{os.path.splitext(filename)[0]}.json')
        with open(output_file_path, 'w') as json_file:
            json.dump(data, json_file)        

  0%|          | 5/16096 [00:05<4:38:36,  1.04s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01hyzgaf4ten29dwzg398849s2` on tokens per minute (TPM): Limit 5000, Used 2058, Requested ~10114. Please try again in 1m26.064s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [13]:
response_content

ChatCompletion(id='chatcmpl-e23a7b7d-9d53-4f92-856c-6fe866028287', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n"importance score": 1,\n"reason": "The case imposes new clarification of the law. The dissenting opinion raises significant issues regarding the effectiveness and adequacy of the investigations, suggesting that they were inadequate, superficial, and too late to provide any prospects of solving the murder. This constitutes a significant contribution to the development of the case law concerning Article 2 of the Convention."\n}', role='assistant', function_call=None, tool_calls=None))], created=1717699217, model='mixtral-8x7b-32768', object='chat.completion', system_fingerprint='fp_c5f20b5bb1', usage=CompletionUsage(completion_tokens=93, prompt_tokens=1942, total_tokens=2035, completion_time=0.165629544, prompt_time=1.229387322, queue_time=None, total_time=1.395016866), x_groq={'id': 'req_01hzqdm0mtftsvf01n05zyky83'})

In [75]:
d = response_content.choices[0].message.content

In [76]:
d

'{\n  "importance score": 1,\n  "reason": "The case declares the application admissible, holds that there has been a violation of Article 6 1 of the Convention, and holds that the respondent State is to pay the applicant, within three months from the date on which the judgment becomes final, EUR 7,000 in respect of non-pecuniary damage. This indicates a significant contribution to the development, clarification or modification of the case law as it interprets Article 6 1 of the Convention and sets a financial award."\n}'

In [78]:
d = json.loads(d)
d.keys()

dict_keys(['importance score', 'reason'])

In [79]:
d

{'importance score': 1,
 'reason': 'The case declares the application admissible, holds that there has been a violation of Article 6 1 of the Convention, and holds that the respondent State is to pay the applicant, within three months from the date on which the judgment becomes final, EUR 7,000 in respect of non-pecuniary damage. This indicates a significant contribution to the development, clarification or modification of the case law as it interprets Article 6 1 of the Convention and sets a financial award.'}